In [1]:
import argparse
from pathlib import Path
import pickle
import mlflow

In [2]:
# Define Arguments for this step

class MyArguments:
    def __init__(self, /, **kwargs):
        self.__dict__.update(kwargs)

args = MyArguments(
                model_name = "taxi-model",
                model_path = "/tmp/train",
                evaluation_output = "/tmp/evaluate"
                )

In [3]:
def main(args):
    '''Loads model, registers it if deply flag is True'''

    with open((Path(args.evaluation_output) / "deploy_flag"), 'rb') as infile:
        deploy_flag = int(infile.read())
        
    mlflow.log_metric("deploy flag", int(deploy_flag))
    if deploy_flag==1:

        print("Registering ", args.model_name)

        # load model
        with open((Path(args.model_path) / "model.pkl"), "rb") as infile:
            model = pickle.load(infile)

        # log model using mlflow
        mlflow.sklearn.log_model(model, args.model_name)

        # register logged model using mlflow
        run_id = mlflow.active_run().info.run_id
        model_uri = f'runs:/{run_id}/{args.model_name}'
        mlflow.register_model(model_uri, args.model_name)

    else:
        print("Model will not be registered!")

In [4]:
mlflow.start_run()

lines = [
    f"Model name: {args.model_name}",
    f"Model path: {args.model_path}",
    f"Evaluation output path: {args.evaluation_output}",
]

for line in lines:
    print(line)

main(args)

mlflow.end_run()

Model name: taxi-model
Model path: /tmp/train
Evaluation output path: /tmp/evaluate
Registering  taxi-model


Registered model 'taxi-model' already exists. Creating a new version of this model...
2022/06/30 12:15:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: taxi-model, version 25
Created version '25' of model 'taxi-model'.
